In [1]:
import os
import glob
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms, utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D')
from efficientnet_pytorch_3d import EfficientNet3D

In [3]:
img_size = 256

In [4]:
def load_image(scan_id, split='train', path='../input/rsna-miccai-png'):
    flair = sorted(glob.glob(f"{path}/{split}/{scan_id}/FLAIR/*.png"))
    t1w = sorted(glob.glob(f"{path}/{split}/{scan_id}/T1w/*.png"))
    t1wce = sorted(glob.glob(f"{path}/{split}/{scan_id}/T1wCE/*.png"))
    t2w = sorted(glob.glob(f"{path}/{split}/{scan_id}/T2w/*.png"))
    
    write = 0
    
    for i in [flair, t1w, t1wce, t2w]:
        
        if len(i) == 0:
            i_img = np.zeros((img_size, img_size, 50))
        elif len(i) < 50:
            i_img = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size))
                            for path in i[0:50]]).T 
            num_zero = 50 - i_img.shape[-1]
            i_img = np.concatenate((i_img, np.zeros((img_size, img_size, num_zero))), -1)
        else: 
            i_img = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size)) 
                             for path in i[len(i) // 2 - 25: len(i) // 2 + 25]]).T
        
        if write == 0: 
            img = i_img 
            write = 1
        else: 
            img = np.concatenate((img, i_img), axis=-1)
            
    return img

In [5]:
class RNSADataset(Dataset):
    def __init__(self, csv, mode='train', transform=None):
        self.csv = csv
        self.mode = mode
        self.transform = transform
        
    def __len__(self): 
        return self.csv.shape[0]
    
    def __getitem__(self, idx): 
        data = self.csv.iloc[idx]
        brat = str(int(data["BraTS21ID"])).zfill(5)
        mgmt = data['MGMT_value']
        
        image = load_image(brat, self.mode)
        
        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image']
        else:
            image = image
        
        image = image.transpose(2, 0, 1)

        if self.mode == 'train': 
            return torch.tensor(image, dtype=torch.float32), torch.tensor(mgmt, dtype=torch.long)
        else: 
            return torch.tensor(image, dtype=torch.float32), brat
            

In [6]:
train_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
train_dataset = RNSADataset(train_df)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=2, num_workers=16)

In [7]:
model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
n_epochs = 10

In [8]:
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
best_pres = 10000

for epoch in range(n_epochs): 
    train_loss = []
    model.train()
    
    for i, data in tqdm(enumerate(train_loader, 0)): 
        x, y = data
        x = torch.unsqueeze(x, dim=1)
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        
        outputs = model(x)
        loss = criterion(outputs, y)
        optimizer.step()
        train_loss.append(loss.item())
    avg_train = sum(train_loss) / len(train_loss)
    print(f'epoch {epoch + 1} train: {avg_train}')
    
    if avg_train < best_pres: 
        print('save model...')
        best_pres = avg_train
        model_name = f'best_loss_{epoch}.pt'
        torch.save(model.state_dict(), f'best_loss_{epoch}.pt')

0it [00:00, ?it/s]

epoch 1 train: 0.7022217855925446
save model...


0it [00:00, ?it/s]

epoch 2 train: 0.6922288559402622
save model...


0it [00:00, ?it/s]

...

epoch 8 train: 0.6986720631147001


0it [00:00, ?it/s]

epoch 9 train: 0.7035089851239436


0it [00:00, ?it/s]

epoch 10 train: 0.6962984804407322


In [9]:
model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
model.to(device)
checkpoint = torch.load(model_name)
model.load_state_dict(checkpoint)
model.eval()

EfficientNet3D(
  (_conv_stem): Conv3dStaticSamePadding(
    1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock3D(
      (_depthwise_conv): Conv3dStaticSamePadding(
        32, 32, kernel_size=(3, 3, 3), stride=[2, 2, 2], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(0, 1, 0, 1, 0, 1), value=0.0)
      )
      (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv3dStaticSamePadding(
        32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv3dStaticSamePadding(
        8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv3dStaticS

In [10]:
test_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
test_dataset = RNSADataset(test_df, mode="test")
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=16)

In [11]:
y_pred = []
ids = []

for e, batch in enumerate(test_loader):
    print(f"{e + 1}/{len(test_loader)}", end="\r")
    batch, brat = batch
    with torch.no_grad():
        tmp_pred = np.zeros((batch.shape[0], 2))
        tmp_res = torch.sigmoid(model(batch.unsqueeze(1).to(device))).cpu().numpy().squeeze()
        tmp_pred += tmp_res
        y_pred.extend(tmp_pred)
        ids.extend([brat])

In [16]:
test = np.array(y_pred)
test_df = pd.DataFrame({"BraTS21ID": test_df['BraTS21ID'].apply(lambda x: str(x).zfill(5)), "MGMT_value": test[:,1]})

In [19]:
test_df.to_csv("submission.csv", index=False)

In [1]:
# todo: data augmentation
# schedule lr
# have a val, and then using roc_auc_score